# 1. Project info

**Project title**: Identify patients with suspected infection using the electronic health record.
<!---Max 41 characters.-->

**Name:** JoAnn Alvarez

**E-mail:** joannmrudd@hotmail.com

**GitHub username**: ruddjm

**Link to prior writing**: https://github.com/ruddjm/data.table/blob/master/dataDotTable.ipynb ; Several posted here: http://biostat.mc.vanderbilt.edu/wiki/Main/JoAnnAlvarez
<!---Please supply a link/reference to some of your own already published educational writing. For example, in the form of a blog post, notebook, article, book or internal case study.-->

**Short description**: Flex your data.table muscle to classify patients with suspected infection using their electronic health record. <!--(111 chr. 110 limit.)-->

#### Long description ####

Sepsis is a deadly illness accounting for a large portion of in-hospital deaths. It occurs when a person's organs shut down in response to a severe infection. This public health problem is a major target for research, and hospital records represent a great opportunity for sepsis research. In this R project, you will identify hospital patients with severe infection using medical record data. This project assumes you know how to work with data frames using `data.table`.

<!---A longer description of the project, around four sentences in length. 
This will be read by the students on the DataCamp platform **before** deciding to start the project. It should mention some of the major prerequisites for completing the project (for example "familiarity with data frame" or "know how to use the `lm` function")-->

#### Datasets used ####

Two dataframes I simulated in R to mimic the format of tables from EHR databases. Medication administration records and blood culture records are in `antibioticDT` and `bloodcultureDT`. The code and R data are located at https://github.com/ruddjm/datacamp_projects_identify_infection_ehr. I envision these data frames already being loaded in the workspace.

<!---Short description (and ideally links) to the datasets used in the project. This will be read my me (Rasmus) only.-->

#### Assumed student background ####

   * Students should have some experience using R and have been introduced to `data.table`. 
      * They should understand how indexing works in R (`[]`). 
      * `merge` function in R. They need to understand the concept of merging, inner joins, left inner joins, and `by` variables (but not necessarily the `join` terminology). 
      * Assignment in `data.table` using ":="
      * Grouped aggregations in `data.table`
      * `shift` function in `data.table`
      
This should not require any knowledge of healthcare or medicine. 

<!---What background knowledge you assume the student doing this project will have. The more specific the better. This will be read my me (Rasmus) only. Please list things like modules, tools, functions, methods, and statistical concepts and jargon.

Not so useful: "The student has a basic familiarity with data frames."

More useful: "The student knows how to read in a csv file as a data frame and how to compute grouped summary statistics using `dplyr`."-->

---
# Example Project info (feel free to delete)

**Project title**: The discovery of handwashing

**Name:** Rasmus Bååth

**E-mail:** rasmus.baath@gmail.com

**GitHub username**: rasmusab

**Link to prior writing**: http://sumsar.net/blog/2016/09/whats-on-the-menu/

**Short description**: Reanalyse the data behind one of the most important discoveries of modern medicine: Handwashing.

#### Long description ####

In 1847 the Hungarian physician Ignaz Semmelweis makes a breakthrough discovery: He discovers handwashing. Contaminated hands was a major cause of childbed fever and by enforcing handwashing at his hospital he saved hundreds of lives. In this R project, we will reanalyze the medical data Semmelweis collected. This project assumes you have familiarity handling data frames and knows how to perform a bootstrap analysis. 

#### Datasets used ####

This project uses the original data collected by Dr. Semmelweis which should be in the public domain. I haven't found any tidy `csv` files, so I scraped the data from this Wikipedia page: https://en.wikipedia.org/wiki/Historical_mortality_rates_of_puerperal_fever

`yearly_deaths_by_clinic.csv`: https://gist.github.com/rasmusab/a207ab6ce2af0d74d20a3559a4ad3d9b

`monthly_deaths.csv`: https://gist.github.com/rasmusab/02c06684ca8fd93b5125b713ffe2b9f6

#### Assumed student background ####

The student can handle basic R control structures like `if` statements and `for` loops. The student can load in and inspect `csv` files. She knows how to select column and subset rows using the `[]` and `$` operators and understand vectorized operators (like that `+` is vectorized). The student has produced line plots using the `plot` function. The student knows that a bootstrap distribution is created by repeatedly resampling a dataset, and knows how to calculate a bootstrap confidence interval using the `quantile` function.

---

# 2. Project narrative intro

## 1. This patient may have sepsis.

Sepsis is a deadly syndrome where a patient has a severe infection that causes organ failure. The medical community believes that the sooner septic patients are treated, the more likely they are to survive, but recognizing sepsis is difficult. Now that hospitals have a constant flow of data, it may be possible to use machine learning to automatically flag patients who are likely to be septic. Before any predictive algorithms can be developed, we need a reliable way to pick out the patients who had sepsis. 
 
In this project, we will find out which patients were suspected to have severe infection using electronic health record (EHR) data. These criteria are a very simplified version of the ones in a 2017 JAMA article by Rhee. The basic idea is that a patient starts antibiotics within a couple of days of a blood culture, and is given antibiotics for at least 4 days.

### Criteria for Suspected Infection
   * Sequence of ≥4 “Qualifying Antibiotic Days” (QADs)
      * Must start with a “new antibiotic” (not given in the prior 2 days).
      * Must start within +/-2 days of a blood culture.  
      * There must be at least one IV antibiotic within the +/-2 day window period.

Take a look at the antibiotic data. Each row is one drug administration, and includes the patient id, day the drug was administered, and the drug type. First let's find which rows represent 'new' antibiotics, by checking whether they were given in the past 2 days. 

In [1]:
library(data.table)
load('ehr_dataframes.Rdata', verbose = TRUE)

setorder(antibioticDT, id, abx_type, abx_day)
antibioticDT[ , lastAdministrationDay := shift(abx_day, 1), 
  by = .(id, abx_type)]
antibioticDT[ , daysSinceLastAdmin := abx_day - lastAdministrationDay]
antibioticDT[ , antibiotic_new := 1]
antibioticDT[daysSinceLastAdmin <= 2, antibiotic_new := 0]

Warning message:
“package ‘data.table’ was built under R version 3.3.2”

Loading objects:
  antibioticDT
  blood_cultureDT


## 2. Combine the antibiotic data and the blood culture data

To find which antibiotics were given close to a blood culture, we'll need to combine the drug administration data with the blood culture data. 

A tricky part is that many patients will have had blood cultures on several different days. For each one of them, we are going to see if there's a sequence of antibiotic days close to it. For a given patient, we will match each blood culture to all of his antibiotics.

In [4]:
combinedDT = merge(
  blood_cultureDT,
  antibioticDT,
  all = FALSE,
  by = 'id',
  allow.cartesian = TRUE)

## 3. Determine whether each row is in-window.

Make a new variable indicating whether the given row is within two days of a blood culture. 

In [5]:
combinedDT[ , 
  in_bcx_window := 
    abx_day - blood_culture_day <= 2 
    & 
    abx_day - blood_culture_day >= -2]